In [1]:
#IMPORTS
import numpy as np
import pandas as pd
import scanpy as sc
import scanpy.external as sce
# from scipy import io
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
#Loads the Data
sc.settings.verbosity = 3            
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

adata = sc.read_10x_mtx(
    "temp_asc_vaccines/mcm287_A",  # the directory with the `.mtx` file
    var_names='gene_symbols',                # use gene symbols for the variable names (variables-axis index)
    cache=True)
adata.var_names_make_unique()
sample = 'mcm287_A' #sample name
results_file = sample + '_sr.h5ad'  # the file that will store the analysis results
adata.var['mt'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
ribo_genes = adata.var_names.str.startswith(("RPS","RPL"))

#Calculates Percent Ribosomal
adata.obs['percent_ribo'] = np.sum(
    adata[:, ribo_genes].X, axis=1).A1 / np.sum(adata.X, axis=1).A1

#plots violin plot before filtering
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt','percent_ribo'],
             jitter=0.4, multi_panel=True)

In [ ]:
#Gets initial cell count, prints it
initial_cell_count = adata.n_obs
print(f"Initial number of cells: {initial_cell_count}")

# filter by gene count within cells
# sc.pp.filter_cells(adata, min_counts= 4000)

#filter out genes that appear in less than 3 cells
sc.pp.filter_genes(adata, min_cells=3)

#filter out cells with more than 150000
sc.pp.filter_cells(adata, max_counts = 150000)


cell_count = adata.n_obs

#filters with at least n genes with count 1
adata = adata[adata.obs.n_genes_by_counts > 600, :]

filtered_cell_count = adata.n_obs

#filters by percent mitochondrial
adata = adata[adata.obs.pct_counts_mt < 10, :]

filtered_cell_count2 = adata.n_obs

cells_filtered_out = cell_count - filtered_cell_count
cells_filtered_out2 = filtered_cell_count - filtered_cell_count2

#prints out all the filter counts
print(f"Number of cells filtered out in counts filtering: {cells_filtered_out}")
print(f"Number of cells after counts filtering: {filtered_cell_count}")
print(f"Number of cells filtered out in mt filtering: {cells_filtered_out2}")
print(f"Number of cells after mt filtering: {filtered_cell_count2}")

#plots the new violin plot after filtering
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt', 'percent_ribo'],
             jitter=0.4, multi_panel=True)

#adds biomart annotations
annot = sc.queries.biomart_annotations(
    "hsapiens",
    ["ensembl_gene_id", "gene_biotype"], use_cache = True
).set_index('ensembl_gene_id')
adata.var['gene_ids'] = adata.var['gene_ids'].astype(str)
annot.index = annot.index.astype(str)
gene_biotype_dict = annot['gene_biotype'].to_dict()
#adds biotype to adata object
adata.var['gene_biotype'] = adata.var['gene_ids'].map(gene_biotype_dict)
#dict of genes to exclude (in this case, IG genes)
genes_to_exclude = ['IG_C_gene', 'IG_C_pseudogene', 'IG_D_gene', 'IG_D_pseudogene', 'IG_J_gene', 'IG_LV_gene', 'IG_pseudogene', 'IG_V_gene', 'IG_V_pseudogene', 'IG_J_pseudogene']  # Replace with your list of genes
print(adata)
# Create a mask to filter out these genes (mask is genes that aren't in the gene exclusion list)
mask = ~adata.var["gene_biotype"].isin(genes_to_exclude)
# Create a new AnnData object with the filtered genes
adata_filtered = adata.copy()
adata_filtered = adata_filtered[:, mask]

########
#uses the mask to calculate the IG gene expression by summing what isn't in the mask which isn't the list so it sums the list
adata.obs['IG_gene_expression'] = adata[:, ~mask].X.sum(axis=1)

adata.obs['total_gene_expression'] = adata.X.sum(axis=1)


# Calculate the percentage of IG gene expression relative to the total gene expression
adata_filtered.obs['IG_gene_percentage'] = (adata.obs['IG_gene_expression'] / adata.obs['total_gene_expression']) * 100
adata_filtered.write(sample + '_foritgr.h5ad')


########

In [ ]:
sc.pp.normalize_total(adata_filtered, target_sum=1e4)
sc.pp.log1p(adata_filtered)
adata_filtered.raw = adata_filtered
# Find highly variable genes
#finds and plots highly variable genes
sc.pp.highly_variable_genes(adata_filtered, min_mean=0.0125, max_mean=3, min_disp=0.5)

#filters the adata to include only the highly variable genes
adata_filtered = adata_filtered[:, adata_filtered.var.highly_variable]

#regresses out the total counts and mitochondrial counts before doing principal component analysis
sc.pp.regress_out(adata_filtered, ['total_counts', 'pct_counts_mt', 'percent_ribo'])

#Scales each gene to unit variance and clips values exceeding standard deviation 10.
sc.pp.scale(adata_filtered, max_value=10)

#performs principal coponent analysis and plots the variance ratio
sc.tl.pca(adata_filtered, svd_solver='arpack')
sc.pl.pca_variance_ratio(adata_filtered, log=True)

#preserves a copy of the adata object
copydata = adata_filtered.copy()
#calls the standard dimensionality reduction and clustering steps on the adata object 
# neighbors, UMAP, and leiden (clustering)
sc.pp.neighbors(adata_filtered, n_neighbors=10, n_pcs=30)
sc.tl.umap(adata_filtered)

In [ ]:
### leftover scrublet code, use if you want to doublet filter at some point
# sc.external.pp.scrublet(adata_filtered)
# sc.external.pl.scrublet_score_distribution(adata_filtered)
# adata_filtered = (adata_filtered[adata_filtered.obs['predicted_doublet'] == False])

In [ ]:
#clustering resolution should be adjusted to ensure that it matches the elbow plot produced above
#cut off should be around the "elbow" or corner of the plot where intra-PC variance becomes minimal
res = .3
sc.tl.leiden(adata_filtered, resolution= res, key_added = "leiden")

#plots the umap based on both sample and cluster
sc.pl.umap(adata_filtered, color = ['leiden'], hspace = 1)


adata.obsm['X_umap'] = adata_filtered.obsm['X_umap']

# Sum the expression values on the adata object
adata.obs['IG_gene_percentage'] = (adata.obs['IG_gene_expression'] / adata.obs['total_gene_expression']) * 100

# Plot UMAP and color by the combined expression
sc.pl.umap(adata, color=['IG_gene_expression'])

# Plot UMAP and color by the percentage of IG gene expression
sc.pl.umap(adata, color=['IG_gene_percentage'])


# Extract leiden cluster information
leiden_clusters = adata_filtered.obs['leiden']
cluster_counts = leiden_clusters.value_counts().sort_index()

# Get UMAP colors for each cluster
umap_colors = adata_filtered.uns['leiden_colors']

reformed_counter = len(cluster_counts)
umap_colors = umap_colors[0: reformed_counter]

# Create a DataFrame for easy handling
df = pd.DataFrame({
    'Cluster': cluster_counts.index,
    'Count': cluster_counts.values,
    'Color': umap_colors
})

# Plotting
fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.bar(df['Cluster'], df['Count'], color=df['Color'])

# Adding count values above the bars
for bar in bars:
    yval = bar.get_height()
    ax.text(bar.get_x() + bar.get_width() / 2, yval + 0.5, int(yval), ha='center', va='bottom')

# Customize plot
ax.set_xlabel('Cluster')
ax.set_ylabel('Count')
ax.set_title('Data Count by Leiden Cluster')
ax.set_xticks(np.arange(len(df['Cluster'])))
ax.set_xticklabels(df['Cluster'])

plt.show()

In [ ]:

sc.tl.rank_genes_groups(adata_filtered, 'leiden', method='wilcoxon')
sc.pl.rank_genes_groups(adata_filtered, n_genes=20, sharey=False)
ranks = pd.DataFrame(adata_filtered.uns['rank_genes_groups']['names']).head(50)
ranks.to_csv('rank_genes_groups' + sample + '.csv')
result = adata_filtered.uns['rank_genes_groups']
groups = result['names'].dtype.names
scoresgroup = pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names', 'pvals']}).head(50)
scoresgroup.to_csv('scores' + sample + '.csv')
del adata_filtered.var['gene_biotype']
adata_filtered.write(results_file)


This is extra code left over from a previous project - it takes the data, randomly downsamples it 3 times to 85% of the original size, and then reclusters it all and compares the clusters to see if the data clusters in a similar way. This is essentially a good proxy for the robustness of the clustering, and how variable it can be to small parts of the data.

In [ ]:
adata1 = sc.pp.subsample(copydata, fraction=.85, random_state=1, copy=True)
adata2 = sc.pp.subsample(copydata, fraction=.85, random_state=200, copy=True)
adata3 = sc.pp.subsample(copydata, fraction=.85, random_state=373, copy=True)

sc.pp.neighbors(adata1, n_neighbors=10, n_pcs=40)
sc.tl.umap(adata1)
sc.tl.leiden(adata1, resolution= res, key_added = "leiden")
sc.pl.umap(adata1, color = ['leiden'], hspace = 1, title = "Downsample 1")
plt.show()

sc.pp.neighbors(adata2, n_neighbors=10, n_pcs=40)
sc.tl.umap(adata2)
sc.tl.leiden(adata2, resolution= res, key_added = "leiden")
sc.pl.umap(adata2, color = ['leiden'], hspace = 1, title="Downsample 2")
plt.show()

sc.pp.neighbors(adata3, n_neighbors=10, n_pcs=40)
sc.tl.umap(adata3)
sc.tl.leiden(adata3, resolution= res, key_added = "leiden")
sc.pl.umap(adata3, color = ['leiden'], hspace = 1, title="Downsample 3")
plt.show()
labels1 = adata1.obs['leiden']
labels2 = adata2.obs['leiden']
cmtx = pd.crosstab(labels2, labels1)
ncmtx = cmtx.div(cmtx.sum(axis=1), axis=0)
count_greater_than_08 = (ncmtx >= 0.8).sum().sum()
max_rows_columns = (ncmtx.shape[0])

plt.subplots(figsize=(20,15))
ax = sns.heatmap(ncmtx, annot=True)
ax.set(xlabel = 'random1', ylabel = 'random2', title = (f"1 vs 2 Normalized: Number of clusters with >=80% cell congruency: {count_greater_than_08} / {max_rows_columns}"))
plt.show()
# plt.subplots(figsize=(20,15))
# ax = sns.heatmap(cmtx, annot=True)
# ax.set(xlabel = 'random1', ylabel = 'random2', title = "1 vs 2 Raw")
# plt.show()

labels1 = adata1.obs['leiden']
labels2 = adata3.obs['leiden']
cmtx = pd.crosstab(labels2, labels1)
ncmtx = cmtx.div(cmtx.sum(axis=1), axis=0)
count_greater_than_08 = (ncmtx >= 0.8).sum().sum()
max_rows_columns = (ncmtx.shape[0])
plt.subplots(figsize=(20,15))
ax = sns.heatmap(ncmtx, annot=True)
ax.set(xlabel = 'random1', ylabel = 'random3', title = (f"1 vs 3 Normalized: Number of clusters with >=80% cell congruency: {count_greater_than_08} / {max_rows_columns}"))
plt.show()
# plt.subplots(figsize=(20,15))
# ax = sns.heatmap(cmtx, annot=True)
# ax.set(xlabel = 'random1', ylabel = 'random3', title = "1 vs 3 Raw")
# plt.show()

labels1 = adata2.obs['leiden']
labels2 = adata3.obs['leiden']
cmtx = pd.crosstab(labels2, labels1)
ncmtx = cmtx.div(cmtx.sum(axis=1), axis=0)
count_greater_than_08 = (ncmtx >= 0.8).sum().sum()
max_rows_columns = (ncmtx.shape[0])
plt.subplots(figsize=(20,15))
ax = sns.heatmap(ncmtx, annot=True)
ax.set(xlabel = 'random2', ylabel = 'random3', title = (f"2 vs 3 Normalized: Number of clusters with >=80% cell congruency: {count_greater_than_08} / {max_rows_columns}"))
plt.show()
# plt.subplots(figsize=(20,15))
# ax = sns.heatmap(cmtx, annot=True)
# ax.set(xlabel = 'random2', ylabel = 'random3', title = "2 vs 3 Raw")
# plt.show()


labels1 = adata_filtered.obs['leiden']
labels2 = adata1.obs['leiden']
cmtx = pd.crosstab(labels2, labels1)
ncmtx1 = cmtx.div(cmtx.sum(axis=1), axis=0)
count_greater_than_08 = (ncmtx1 >= 0.8).sum().sum()
max_rows_columns = (ncmtx1.shape[0])
plt.subplots(figsize=(20,15))
ax = sns.heatmap(ncmtx1, annot=True)
ax.set(xlabel = 'Full', ylabel = 'random1', title = (f"1 vs Full Normalized: Number of clusters with >=80% cell congruency: {count_greater_than_08} / {max_rows_columns}"))
plt.show()
# plt.subplots(figsize=(20,15))
# ax = sns.heatmap(cmtx, annot=True)
# ax.set(xlabel = 'Full', ylabel = 'random1', title = "1 vs Full Raw")
# plt.show()

labels1 = adata_filtered.obs['leiden']
labels2 = adata2.obs['leiden']
cmtx = pd.crosstab(labels2, labels1)
ncmtx2 = cmtx.div(cmtx.sum(axis=1), axis=0)
count_greater_than_08 = (ncmtx2 >= 0.8).sum().sum()
max_rows_columns = (ncmtx2.shape[0])
plt.subplots(figsize=(20,15))
ax = sns.heatmap(ncmtx2, annot=True)
ax.set(xlabel = 'Full', ylabel = 'random2', title = (f"2 vs Full Normalized: Number of clusters with >=80% cell congruency: {count_greater_than_08} / {max_rows_columns}"))
plt.show()
# plt.subplots(figsize=(20,15))
# ax = sns.heatmap(cmtx, annot=True)
# ax.set(xlabel = 'Full', ylabel = 'random2', title = "2 vs Full Raw")
# plt.show()

labels1 = adata_filtered.obs['leiden']
labels2 = adata3.obs['leiden']
cmtx = pd.crosstab(labels2, labels1)
ncmtx3 = cmtx.div(cmtx.sum(axis=1), axis=0)
count_greater_than_08 = (ncmtx3 >= 0.8).sum().sum()
max_rows_columns = (ncmtx3.shape[0])
#max_rows_columns = max(ncmtx3.shape[0], ncmtx3.shape[1])
plt.subplots(figsize=(20,15))
ax = sns.heatmap(ncmtx3, annot=True)
ax.set(xlabel = 'Full', ylabel = 'random3', title = (f"3 vs Full Normalized: Number of clusters with >=80% cell congruency: {count_greater_than_08} / {max_rows_columns}"))
plt.show()
# plt.subplots(figsize=(20,15))
# ax = sns.heatmap(cmtx, annot=True)
# ax.set(xlabel = 'Full', ylabel = 'random3', title = "3 vs Full Raw")
# plt.show()


x = 0
ncmtx_list = [ncmtx1, ncmtx2, ncmtx3]
data_frames_dict = {}
while x <= 2:
    # Define the threshold for keeping cells
    threshold = 0.8
    ncmtx = ncmtx_list[x]

    # Identify clusters with scores greater than or equal to 0.8
    clusters_above_threshold = ncmtx.columns[ncmtx.max(axis=0) >= threshold]

    # Create a DataFrame to store selected clusters and row values
    selected_clusters_df = pd.DataFrame(index=ncmtx.index, columns=['Row'] + clusters_above_threshold.tolist())

    # Fill the 'Row' column with row values
    selected_clusters_df['Row'] = selected_clusters_df.index

    # Fill the DataFrame with column labels for rows with scores above the threshold
    for cluster in clusters_above_threshold:
        selected_clusters_df[cluster] = ncmtx[cluster].apply(lambda x: cluster if x >= threshold else None)

    # Melt the DataFrame to have rows first, then columns
    selected_clusters_df = selected_clusters_df.melt(id_vars=['Row'], value_vars=clusters_above_threshold, var_name='Cluster')

    # Drop rows with NaN values
    selected_clusters_df = selected_clusters_df.dropna()

    # Reset index for the final result
    selected_clusters_df.reset_index(drop=True, inplace=True)
    selected_clusters_df = selected_clusters_df[['Row', 'Cluster']]

    rows_below_threshold = ncmtx.index.difference(selected_clusters_df['Row'])

    # Iterate over rows below the threshold and find additional clusters
    for row in rows_below_threshold:
        # Sort clusters by score in descending order
        sorted_clusters = ncmtx.loc[row].sort_values(ascending=False)
        
        # Find the minimum number of clusters to reach a combined score of at least 0.8
        cumulative_sum = 0
        min_clusters = []
        
        for cluster, score in sorted_clusters.items():
            cumulative_sum += score
            min_clusters.append(cluster)
            
            if cumulative_sum >= threshold:
                break

        # Append the row value and selected clusters to the existing DataFrame
        additional_clusters_df = pd.DataFrame({'Row': [row], 'Cluster': [min_clusters]})
        selected_clusters_df = pd.concat([selected_clusters_df, additional_clusters_df], ignore_index=True)

    selected_clusters_df = selected_clusters_df.sort_values(by='Row').reset_index(drop=True)
    data_frames_dict[f'iteration{x}'] =  selected_clusters_df
    x = x + 1
print("Selected Clusters and Column Values:")
print(data_frames_dict)


df_clusters1 = data_frames_dict['iteration0']
df_clusters2 = data_frames_dict['iteration1']
df_clusters3 = data_frames_dict['iteration2']

# Build mappings for each dataframe
cluster_mappings = []
for df_clusters in [df_clusters1, df_clusters2, df_clusters3]:
    mapping = {}
    for index, row in df_clusters.iterrows():
        allowed_clusters = row['Cluster']
        if not isinstance(allowed_clusters, list):
            allowed_clusters = [allowed_clusters]
        mapping[row['Row']] = allowed_clusters
    cluster_mappings.append(mapping)

# Get the cluster labels for each cell in dataset1 and dataset2
clusters_dataset1 = adata1.obs['leiden']  # Replace 'leiden' with your actual cluster column name if different
clusters_dataset2 = adata2.obs['leiden']  # Replace 'leiden' with your actual cluster column name if different
clusters_dataset3 = adata3.obs['leiden']
clusters_dataset0 = adata_filtered.obs['leiden']

reduced_clusters = {1: clusters_dataset1, 2: clusters_dataset2, 3: clusters_dataset3}
# Filter out cells from dataset2 based on the mappings
cells_to_keep = []
for cell in adata2.obs_names:
    correct_count = 0
    total_count = 0
    x = 1
    for mapping in cluster_mappings:
        if cell in reduced_clusters[x].index:
            total_count += 1
            cluster_label_dataset1 = reduced_clusters[x][cell]
            cluster_label_root = clusters_dataset0[cell]
            if cluster_label_root in mapping.get(cluster_label_dataset1, []):
                correct_count += 1
        x = x + 1
    # Keep the cell if it's correct in at least two dataframes or if it appears in less than two dataframes
    if correct_count >= 2 or (total_count < 2 & correct_count == 1):
        cells_to_keep.append(cell)

# Subset dataset2 to only include cells that are in the list of cells to keep
adata0_filtered = adata_filtered[cells_to_keep]

sc.pl.umap(adata0_filtered, color = ['leiden'], hspace = 1, title="Downsample Cluster-Checked Removal")

adata.obs['keep'] = adata.obs_names.isin(cells_to_keep)

# Convert the boolean array to a categorical type for better plotting
adata.obs['keep'] = adata.obs['keep'].astype('category')

# Plot the UMAP
sc.pl.umap(adata, color='keep', palette=['red', 'blue'], title='UMAP of Cells Kept/ Discarded')